In [ ]:
model_id = "symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli"


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
pip install -U sentence-transformers

In [ ]:
plats = "/content/drive/My Drive/Seguimiento-Lucía/borradores/embeddings de literales/resultados_all_writtenReps_plats.csv"
jap = "/content/drive/My Drive/Seguimiento-Lucía/borradores/embeddings de literales/resultados_all_writtenReps_jap.csv"
gold ="/content/drive/My Drive/Seguimiento-Lucía/borradores/embeddings de literales/matches_gold.csv"

In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score

# Cargar los archivos CSV
df_plats = pd.read_csv(plats)  # Reemplaza con tu ruta
df_jap = pd.read_csv(jap)  # Reemplaza con tu ruta

# Cargar el gold-standard
gold_standard = pd.read_csv(gold)  # Reemplaza con tu ruta

# Función para extraer el primer 'writtenRep' y construir la cadena deseada
def construir_cadena(row):
    writtenRep = row['writtenRep'].split(",")[0] if pd.notna(row['writtenRep']) else ''
    prefLabel = row['prefLabel'] if pd.notna(row['prefLabel']) else ''
    return f"{writtenRep} és un plat de {prefLabel}".strip()

# Aplicar la función para construir la cadena a ambos DataFrames
df_plats['text'] = df_plats.apply(construir_cadena, axis=1)
df_jap['text'] = df_jap.apply(construir_cadena, axis=1)

# Usar solo la columna 'text' para generar las oraciones para los embeddings
messages_plats = df_plats["text"].astype(str).tolist()  # Convertir a lista
messages_jap = df_jap["text"].astype(str).tolist()  # Convertir a lista

# Cargar el modelo Universal Sentence Encoder desde TensorFlow Hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

# Generar embeddings usando el Universal Sentence Encoder
embeddings_plats = embed(messages_plats).numpy()
embeddings_jap = embed(messages_jap).numpy()

# Calcular la similitud coseno entre los embeddings
cosine_similarities = cosine_similarity(embeddings_plats, embeddings_jap)

# Establecer un umbral de similitud
umbral = 0.75

# Crear una lista para almacenar los matches detectados
detected_matches = []

# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.shape[0]):
    for j in range(cosine_similarities.shape[1]):
        similarity = cosine_similarities[i, j]
        if similarity > umbral:
            plats_id = df_plats['concepto'].iloc[i]
            jap_id = df_jap['concepto'].iloc[j]
            detected_matches.append((plats_id, jap_id, similarity))

# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim in detected_matches])

# Crear listas para las métricas de evaluación
y_true = []
y_pred = []

# Llenar y_true y y_pred
for plats_id, jap_id in detected_pairs:
    y_true.append(1 if (plats_id, jap_id) in gold_pairs else 0)
    y_pred.append(1)

# Agregar pares no detectados como negativos
for plats_id, jap_id in gold_pairs:
    if (plats_id, jap_id) not in detected_pairs:
        y_true.append(1)  # verdadero positivo
        y_pred.append(0)   # falso negativo

# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Imprimir los matches encontrados con similitud
for plats_id, jap_id, similarity in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")


Detected Matches: 0
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000


In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

# Cargar los archivos CSV
plats_df = pd.read_csv(plats)
jap_df = pd.read_csv(jap)

# Cargar el gold-standard
gold_standard = pd.read_csv(gold)

# Cargar el modelo de embeddings
model_id = "symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli"
model = SentenceTransformer(model_id)

# Crear las secuencias para embeddings (PrefLabel + WrittenRep)
plats_df['sentence'] = plats_df['prefLabel'] + ". " + plats_df['writtenRep']
jap_df['sentence'] = jap_df['prefLabel'] + ". " + jap_df['writtenRep']

# Obtener los embeddings de los archivos (Plats y Gastronomía Japonesa)
plats_embeddings = model.encode(plats_df['sentence'].tolist(), convert_to_tensor=True)
jap_embeddings = model.encode(jap_df['sentence'].tolist(), convert_to_tensor=True)

# Normalizar los embeddings para usar similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular la matriz de similitud coseno
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Establecer un umbral de similitud
umbral = 0.75

# Crear una lista para almacenar los matches detectados
detected_matches = []

# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.size(0)):
    for j in range(cosine_similarities.size(1)):
        similarity = cosine_similarities[i, j].item()
        if similarity > umbral:
            plats_id = plats_df['concepto'].iloc[i]
            jap_id = jap_df['concepto'].iloc[j]
            detected_matches.append((plats_id, jap_id, similarity))

# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim in detected_matches])

# Métricas de evaluación
y_true = [1 if (plats_id, jap_id) in gold_pairs else 0 for plats_id, jap_id in detected_pairs]
y_pred = [1] * len(detected_pairs)  # Todos los matches detectados son positivos

# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Imprimir los matches encontrados con similitud
for plats_id, jap_id, similarity in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

KeyboardInterrupt: 

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Cargar el modelo preentrenado
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Cargar los archivos CSV
plats_df = pd.read_csv(plats)
jap_df = pd.read_csv(jap)

# Cargar el gold-standard
gold_standard = pd.read_csv(gold)

# Función para obtener solo la primera writtenRep
def get_first_writtenrep(row):
    return row['writtenRep'].split(",")[0].strip() if isinstance(row['writtenRep'], str) else ""

# Crear las secuencias para embeddings usando 'prefLabel' y el primer 'writtenRep'
plats_df['sentence'] = plats_df['prefLabel'] + ". " + plats_df.apply(get_first_writtenrep, axis=1)
jap_df['sentence'] = jap_df['prefLabel'] + ". " + jap_df.apply(get_first_writtenrep, axis=1)

# Obtener los embeddings de los archivos (Plats y Gastronomía Japonesa)
plats_embeddings = model.encode(plats_df['sentence'].tolist(), convert_to_tensor=True)
jap_embeddings = model.encode(jap_df['sentence'].tolist(), convert_to_tensor=True)

# Normalizar los embeddings para usar similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular la matriz de similitud coseno
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Establecer un umbral de similitud
umbral = 0.75

# Crear una lista para almacenar los matches detectados
detected_matches = []

# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.size(0)):
    for j in range(cosine_similarities.size(1)):
        similarity = cosine_similarities[i, j].item()
        if similarity > umbral:
            plats_id = plats_df['concepto'].iloc[i]
            jap_id = jap_df['concepto'].iloc[j]
            detected_matches.append((plats_id, jap_id, similarity))

# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim in detected_matches])

# Métricas de evaluación
y_true = [1 if (plats_id, jap_id) in gold_pairs else 0 for plats_id, jap_id in detected_pairs]
y_pred = [1] * len(detected_pairs)  # Todos los matches detectados son positivos

# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Imprimir los matches encontrados con similitud
for plats_id, jap_id, similarity in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Detected Matches: 103
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Plats ID: plats_a_la_carta_C1055 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.7672
Plats ID: plats_a_la_carta_C12 - Jap ID: gastronomia_japonesa_C10 - Similitud: 0.7520
Plats ID: plats_a_la_carta_C12 - Jap ID: gastronomia_japonesa_C5 - Similitud: 0.7711
Plats ID: plats_a_la_carta_C1311 - Jap ID: gastronomia_japonesa_C5 - Similitud: 0.7548
Plats ID: plats_a_la_carta_C1380 - Jap ID: gastronomia_japonesa_C5 - Similitud: 0.7582
Plats ID: plats_a_la_carta_C1471 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.7674
Plats ID: plats_a_la_carta_C1512 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.7548
Plats ID: plats_a_la_carta_C1533 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.7810
Plats ID: plats_a_la_carta_C1556 - Jap ID: gastronomia_japonesa_C18 - Similitud: 0.7665
Plats ID: plats_a_la_carta_C1556 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.7772
Plats ID: plats_a_la_carta_C1598 - Jap ID: gastronomia_japon

In [ ]:
# Crear una lista para almacenar los matches detectados
detected_matches = []

# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.size(0)):
    for j in range(cosine_similarities.size(1)):
        similarity = cosine_similarities[i, j].item()
        if similarity > umbral:
            plats_id = plats_df['concepto'].iloc[i]
            jap_id = jap_df['concepto'].iloc[j]
            detected_matches.append((plats_id, jap_id, similarity))

# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim in detected_matches])

# Calcular el número de matches en el gold standard
matches_in_gold_standard = len([pair for pair in detected_pairs if pair in gold_pairs])

# Total de matches detectados
total_detected_matches = len(detected_pairs)

# Calcular el porcentaje de matches que pertenecen al gold standard
percentage_in_gold_standard = (matches_in_gold_standard / total_detected_matches) * 100 if total_detected_matches > 0 else 0

# Métricas de evaluación
y_true = [1 if (plats_id, jap_id) in gold_pairs else 0 for plats_id, jap_id in detected_pairs]
y_pred = [1] * len(detected_pairs)  # Todos los matches detectados son positivos

# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Matches in Gold Standard: {matches_in_gold_standard}")
print(f"Percentage of Matches in Gold Standard: {percentage_in_gold_standard:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Imprimir los matches encontrados con similitud
for plats_id, jap_id, similarity in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")

Detected Matches: 103
Matches in Gold Standard: 0
Percentage of Matches in Gold Standard: 0.00%
Precision: 0.0000
Recall: 0.0000
F1 Score: 0.0000
Plats ID: plats_a_la_carta_C1055 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.7672
Plats ID: plats_a_la_carta_C12 - Jap ID: gastronomia_japonesa_C10 - Similitud: 0.7520
Plats ID: plats_a_la_carta_C12 - Jap ID: gastronomia_japonesa_C5 - Similitud: 0.7711
Plats ID: plats_a_la_carta_C1311 - Jap ID: gastronomia_japonesa_C5 - Similitud: 0.7548
Plats ID: plats_a_la_carta_C1380 - Jap ID: gastronomia_japonesa_C5 - Similitud: 0.7582
Plats ID: plats_a_la_carta_C1471 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.7674
Plats ID: plats_a_la_carta_C1512 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.7548
Plats ID: plats_a_la_carta_C1533 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.7810
Plats ID: plats_a_la_carta_C1556 - Jap ID: gastronomia_japonesa_C18 - Similitud: 0.7665
Plats ID: plats_a_la_carta_C1556 - Jap ID: gastronomia_japonesa_C3 - Simi

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Cargar el modelo preentrenado
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Cargar los archivos CSV
plats_df = pd.read_csv(plats)
jap_df = pd.read_csv(jap)

# Cargar el gold-standard
gold_standard = pd.read_csv(gold)

# Función para obtener solo la primera writtenRep
def get_first_writtenrep(row):
    return row['writtenRep'].split(",")[0].strip() if isinstance(row['writtenRep'], str) else ""

# Función para crear una oración con contexto
def create_sentence(row):
    first_writtenrep = get_first_writtenrep(row)
    # Crear la oración con contexto
    return f"{first_writtenrep} és un plat" if first_writtenrep else None

# Crear las secuencias para embeddings usando 'prefLabel' y el primer 'writtenRep' con contexto
plats_df['sentence'] = plats_df['prefLabel'] + ". " + plats_df.apply(create_sentence, axis=1)
jap_df['sentence'] = jap_df['prefLabel'] + ". " + jap_df.apply(create_sentence, axis=1)

# Obtener los embeddings de los archivos (Plats y Gastronomía Japonesa)
plats_embeddings = model.encode(plats_df['sentence'].tolist(), convert_to_tensor=True)
jap_embeddings = model.encode(jap_df['sentence'].tolist(), convert_to_tensor=True)

# Normalizar los embeddings para usar similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular la matriz de similitud coseno
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Establecer un umbral de similitud
umbral = 0.75

# Crear una lista para almacenar los matches detectados
detected_matches = []

# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.size(0)):
    for j in range(cosine_similarities.size(1)):
        similarity = cosine_similarities[i, j].item()
        if similarity > umbral:
            plats_id = plats_df['concepto'].iloc[i]
            jap_id = jap_df['concepto'].iloc[j]
            detected_matches.append((plats_id, jap_id, similarity))

# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim in detected_matches])

# Calcular el número de matches en el gold standard
matches_in_gold_standard = len([pair for pair in detected_pairs if pair in gold_pairs])

# Total de matches detectados
total_detected_matches = len(detected_pairs)

# Calcular el porcentaje de matches que pertenecen al gold standard
percentage_in_gold_standard = (matches_in_gold_standard / total_detected_matches) * 100 if total_detected_matches > 0 else 0

# Métricas de evaluación
y_true = [1 if (plats_id, jap_id) in gold_pairs else 0 for plats_id, jap_id in detected_pairs]
y_pred = [1] * len(detected_pairs)  # Todos los matches detectados son positivos

# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Mostrar resultados
#print(f"Detected Matches: {len(detected_pairs)}")
print(f"Matches in Gold Standard: {matches_in_gold_standard}")
print(f"Percentage of Matches in Gold Standard: {percentage_in_gold_standard:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Imprimir los matches encontrados con similitud
for plats_id, jap_id, similarity in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Streaming output truncated to the last 5000 lines.
Plats ID: plats_a_la_carta_C4732 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.8347
Plats ID: plats_a_la_carta_C4732 - Jap ID: gastronomia_japonesa_C4 - Similitud: 0.8322
Plats ID: plats_a_la_carta_C4732 - Jap ID: gastronomia_japonesa_C6 - Similitud: 0.8172
Plats ID: plats_a_la_carta_C4732 - Jap ID: gastronomia_japonesa_C7 - Similitud: 0.7989
Plats ID: plats_a_la_carta_C4733 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.8111
Plats ID: plats_a_la_carta_C4733 - Jap ID: gastronomia_japonesa_C4 - Similitud: 0.7910
Plats ID: plats_a_la_carta_C4733 - Jap ID: gastronomia_japonesa_C6 - Similitud: 0.7826
Plats ID: plats_a_la_carta_C4734 - Jap ID: gastronomia_japonesa_C3 - Similitud: 0.8420
Plats ID: plats_a_la_carta_C4734 - Jap ID: gastronomia_japonesa_C4 - Similitud: 0.8292
Plats ID: plats_a_la_carta_C4734 - Jap ID: gastronomia_japonesa_C6 - Similitud: 0.8230
Plats ID: plats_a_la_carta_C4735 - Jap ID: gastronomia_japonesa_C3 - Similitud:

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from sklearn.metrics import precision_score, recall_score, f1_score

# Cargar el modelo preentrenado
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# Cargar los archivos CSV
plats_df = pd.read_csv(plats)
jap_df = pd.read_csv(jap)

# Cargar el gold-standard
gold_standard = pd.read_csv(gold)

# Función para obtener solo la primera writtenRep
def get_first_writtenrep(row):
    return row['writtenRep'].split(",")[0].strip() if isinstance(row['writtenRep'], str) else ""

# Función para crear una oración con contexto
def create_sentence(row):
    first_writtenrep = get_first_writtenrep(row)
    # Crear la oración con contexto
    return f"{first_writtenrep} és un plat" if first_writtenrep else None

# Crear las secuencias para embeddings usando 'prefLabel' y el primer 'writtenRep' con contexto
plats_df['sentence'] = plats_df['prefLabel'] + ". " + plats_df.apply(create_sentence, axis=1)
jap_df['sentence'] = jap_df['prefLabel'] + ". " + jap_df.apply(create_sentence, axis=1)

# Obtener los embeddings de los archivos (Plats y Gastronomía Japonesa)
plats_embeddings = model.encode(plats_df['sentence'].tolist(), convert_to_tensor=True)
jap_embeddings = model.encode(jap_df['sentence'].tolist(), convert_to_tensor=True)

# Normalizar los embeddings para usar similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular la matriz de similitud coseno
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Establecer un umbral de similitud
umbral = 0.75

# Crear una lista para almacenar los matches detectados
detected_matches = []

# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.size(0)):
    for j in range(cosine_similarities.size(1)):
        similarity = cosine_similarities[i, j].item()
        if similarity > umbral:
            plats_id = plats_df['concepto'].iloc[i]
            jap_id = jap_df['concepto'].iloc[j]
            detected_matches.append((plats_id, jap_id, similarity))

# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim in detected_matches])

# Calcular el número de matches en el gold standard
matches_in_gold_standard = len([pair for pair in detected_pairs if pair in gold_pairs])

# Total de matches detectados
total_detected_matches = len(detected_pairs)

# Calcular el porcentaje de matches que superan el umbral
matches_above_threshold = len(detected_matches)  # Matches detectados que superan el umbral
percentage_above_threshold = (matches_above_threshold / total_detected_matches) * 100 if total_detected_matches > 0 else 0

# Calcular el porcentaje de matches que pertenecen al gold standard
percentage_in_gold_standard = (matches_in_gold_standard / total_detected_matches) * 100 if total_detected_matches > 0 else 0

# Métricas de evaluación
y_true = [1 if (plats_id, jap_id) in gold_pairs else 0 for plats_id, jap_id in detected_pairs]
y_pred = [1] * len(detected_pairs)  # Todos los matches detectados son positivos

# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred, zero_division=0)
recall = recall_score(y_true, y_pred, zero_division=0)
f1 = f1_score(y_true, y_pred, zero_division=0)

# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Matches Above Threshold: {len(matches_above_threshold)}")
print(f"Percentage of Matches Above Threshold: {percentage_above_threshold:.2f}%")
print(f"Matches in Gold Standard: {matches_in_gold_standard}")
print(f"Percentage of Matches in Gold Standard: {percentage_in_gold_standard:.2f}%")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Imprimir los matches encontrados con similitud
for plats_id, jap_id, similarity in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Detected Matches: 11298


TypeError: object of type 'int' has no len()

In [ ]:
# Cargar los archivos CSV
plats_df = pd.read_csv(plats)
jap_df = pd.read_csv(jap)

# Cargar el gold-standard
gold_standard = pd.read_csv(gold)


# Crear las secuencias para embeddings usando solo 'writtenRep'
plats_df['sentence'] = plats_df['writtenRep']  # Solo usar writtenRep
jap_df['sentence'] = jap_df['writtenRep']      # Solo usar writtenRep

# Obtener los embeddings de los archivos (Plats y Gastronomía Japonesa)
plats_embeddings = model.encode(plats_df['sentence'].tolist(), convert_to_tensor=True)
jap_embeddings = model.encode(jap_df['sentence'].tolist(), convert_to_tensor=True)

# Normalizar los embeddings para usar similitud coseno
plats_embeddings_norm = torch.nn.functional.normalize(plats_embeddings, p=2, dim=1)
jap_embeddings_norm = torch.nn.functional.normalize(jap_embeddings, p=2, dim=1)

# Calcular la matriz de similitud coseno
cosine_similarities = torch.mm(plats_embeddings_norm, jap_embeddings_norm.T)

# Establecer un umbral de similitud
umbral = 0.75

# Crear una lista para almacenar los matches detectados
detected_matches = []

# Buscar los matches con similitud mayor al umbral
for i in range(cosine_similarities.size(0)):
    for j in range(cosine_similarities.size(1)):
        similarity = cosine_similarities[i, j].item()
        if similarity > umbral:
            plats_id = plats_df['concepto'].iloc[i]
            jap_id = jap_df['concepto'].iloc[j]
            detected_matches.append((plats_id, jap_id, similarity))

# Evaluar los resultados frente al gold-standard
gold_pairs = set(zip(gold_standard['Plats_a_la_carta_ID'], gold_standard['Gastronomia_japonesa_ID']))
detected_pairs = set([(plats_id, jap_id) for plats_id, jap_id, sim in detected_matches])

# Métricas de evaluación
y_true = [1 if (plats_id, jap_id) in gold_pairs else 0 for plats_id, jap_id in detected_pairs]
y_pred = [1] * len(detected_pairs)  # Todos los matches detectados son positivos

# Calcular precisión, recall, F1
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Mostrar resultados
print(f"Detected Matches: {len(detected_pairs)}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

# Imprimir los matches encontrados con similitud
for plats_id, jap_id, similarity in detected_matches:
    print(f"Plats ID: {plats_id} - Jap ID: {jap_id} - Similitud: {similarity:.4f}")


Detected Matches: 12
Precision: 0.1667
Recall: 1.0000
F1 Score: 0.2857
Plats ID: plats_a_la_carta_C2056 - Jap ID: gastronomia_japonesa_C15 - Similitud: 0.8015
Plats ID: plats_a_la_carta_C4329 - Jap ID: gastronomia_japonesa_C18 - Similitud: 0.8102
Plats ID: plats_a_la_carta_C4367 - Jap ID: gastronomia_japonesa_C16 - Similitud: 0.8126
Plats ID: plats_a_la_carta_C4367 - Jap ID: gastronomia_japonesa_C17 - Similitud: 0.7786
Plats ID: plats_a_la_carta_C4925 - Jap ID: gastronomia_japonesa_C16 - Similitud: 0.7520
Plats ID: plats_a_la_carta_C5043 - Jap ID: gastronomia_japonesa_C5 - Similitud: 0.9616
Plats ID: plats_a_la_carta_C5775 - Jap ID: gastronomia_japonesa_C18 - Similitud: 0.7874
Plats ID: plats_a_la_carta_C6282 - Jap ID: gastronomia_japonesa_C18 - Similitud: 0.7612
Plats ID: plats_a_la_carta_C6513 - Jap ID: gastronomia_japonesa_C15 - Similitud: 0.7892
Plats ID: plats_a_la_carta_C6528 - Jap ID: gastronomia_japonesa_C10 - Similitud: 0.8111
Plats ID: plats_a_la_carta_C6952 - Jap ID: gastron

In [ ]:
# Matches detectados
detected_matches = [
    ("plats_a_la_carta_C2056", "gastronomia_japonesa_C15", 0.8015),
    ("plats_a_la_carta_C4329", "gastronomia_japonesa_C18", 0.8102),
    ("plats_a_la_carta_C4367", "gastronomia_japonesa_C16", 0.8126),
    ("plats_a_la_carta_C4367", "gastronomia_japonesa_C17", 0.7786),
    ("plats_a_la_carta_C4925", "gastronomia_japonesa_C16", 0.7520),
    ("plats_a_la_carta_C5043", "gastronomia_japonesa_C5", 0.9616),
    ("plats_a_la_carta_C5775", "gastronomia_japonesa_C18", 0.7874),
    ("plats_a_la_carta_C6282", "gastronomia_japonesa_C18", 0.7612),
    ("plats_a_la_carta_C6513", "gastronomia_japonesa_C15", 0.7892),
    ("plats_a_la_carta_C6528", "gastronomia_japonesa_C10", 0.8111),
    ("plats_a_la_carta_C6952", "gastronomia_japonesa_C15", 0.8831),
    ("plats_a_la_carta_C7179", "gastronomia_japonesa_C16", 0.7656),
]

# Gold standard
gold_standard = [
    ("plats_a_la_carta_C6528", "gastronomia_japonesa_C10"),
    ("plats_a_la_carta_C6648", "gastronomia_japonesa_C12"),
    ("plats_a_la_carta_C6911", "gastronomia_japonesa_C14"),
    ("plats_a_la_carta_C6952", "gastronomia_japonesa_C15"),
    ("plats_a_la_carta_C7091", "gastronomia_japonesa_C16"),
    ("plats_a_la_carta_C5043", "gastronomía_japonesa_C5"),
    ("plats_a_la_carta_C2926", "gastronomia_japonesa_C1"),
    ("plats_a_la_carta_C7757", "gastronomia_japonesa_C20"),
    ("plats_a_la_carta_C6513", "gastronomia_japonesa_C9"),
]

# Convertir los datos a sets para una comparación rápida
detected_set = set([(plats_id, jap_id) for plats_id, jap_id, _ in detected_matches])
gold_standard_set = set(gold_standard)

# Obtener los matches correctos
correct_matches = detected_set & gold_standard_set  # Intersección de ambos sets

# Imprimir resultados
print(f"Total de matches correctos: {len(correct_matches)}")
print("Matches correctos:")
for match in correct_matches:
    print(match)

# Si quieres ver qué matches no fueron correctos:
incorrect_matches = detected_set - gold_standard_set
print(f"\nTotal de matches incorrectos: {len(incorrect_matches)}")
print("Matches incorrectos:")
for match in incorrect_matches:
    print(match)


Total de matches correctos: 2
Matches correctos:
('plats_a_la_carta_C6952', 'gastronomia_japonesa_C15')
('plats_a_la_carta_C6528', 'gastronomia_japonesa_C10')

Total de matches incorrectos: 10
Matches incorrectos:
('plats_a_la_carta_C2056', 'gastronomia_japonesa_C15')
('plats_a_la_carta_C4925', 'gastronomia_japonesa_C16')
('plats_a_la_carta_C6282', 'gastronomia_japonesa_C18')
('plats_a_la_carta_C5043', 'gastronomia_japonesa_C5')
('plats_a_la_carta_C4367', 'gastronomia_japonesa_C17')
('plats_a_la_carta_C5775', 'gastronomia_japonesa_C18')
('plats_a_la_carta_C7179', 'gastronomia_japonesa_C16')
('plats_a_la_carta_C4367', 'gastronomia_japonesa_C16')
('plats_a_la_carta_C6513', 'gastronomia_japonesa_C15')
('plats_a_la_carta_C4329', 'gastronomia_japonesa_C18')


In [ ]:
# Matches detectados
detected_matches = [
    ("plats_a_la_carta_C2056", "gastronomia_japonesa_C15", 0.7856),
    ("plats_a_la_carta_C6513", "gastronomia_japonesa_C15", 0.7639),
    ("plats_a_la_carta_C6911", "gastronomia_japonesa_C9", 0.7535),
    ("plats_a_la_carta_C6952", "gastronomia_japonesa_C15", 0.8363),
    ("plats_a_la_carta_C7757", "gastronomia_japonesa_C9", 0.7525),
]

# Gold standard
gold_standard = [
    ("plats_a_la_carta_C6528", "gastronomia_japonesa_C10"),
    ("plats_a_la_carta_C6648", "gastronomia_japonesa_C12"),
    ("plats_a_la_carta_C6911", "gastronomia_japonesa_C14"),
    ("plats_a_la_carta_C6952", "gastronomia_japonesa_C15"),
    ("plats_a_la_carta_C7091", "gastronomia_japonesa_C16"),
    ("plats_a_la_carta_C5043", "gastronomía_japonesa_C5"),
    ("plats_a_la_carta_C2926", "gastronomia_japonesa_C1"),
    ("plats_a_la_carta_C7757", "gastronomia_japonesa_C20"),
    ("plats_a_la_carta_C6513", "gastronomia_japonesa_C9"),
]

# Convertir los datos a sets para una comparación rápida
detected_set = set([(plats_id, jap_id) for plats_id, jap_id, _ in detected_matches])
gold_standard_set = set(gold_standard)

# Obtener los matches correctos
correct_matches = detected_set & gold_standard_set  # Intersección de ambos sets

# Calcular el porcentaje de matches correctos
num_correct_matches = len(correct_matches)
num_total_detected_matches = len(detected_matches)

# Evitar división por cero
if num_total_detected_matches > 0:
    porcentaje_correcto = (num_correct_matches / num_total_detected_matches) * 100
else:
    porcentaje_correcto = 0

# Imprimir resultados
print(f"Total de matches correctos: {num_correct_matches}")
print(f"Total de matches detectados: {num_total_detected_matches}")
print(f"Porcentaje de matches correctos: {porcentaje_correcto:.2f}%")


Total de matches correctos: 1
Total de matches detectados: 5
Porcentaje de matches correctos: 20.00%


In [ ]:
texts = []

# Abrir el archivo de texto en modo lectura
with open(plats, 'r', encoding='utf-8') as archivo:
    # Leer cada línea del archivo
    for linea in archivo:
        # Eliminar espacios innecesarios al inicio y al final de la línea
        linea = linea.strip()
        # Agregar la línea formateada a la lista texts
        texts.append(f'{linea}')

In [ ]:
x = len(texts)
print(x)

7761


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(model_id)
embeddings = model.encode(texts)
print(embeddings)

[[-0.17200027 -0.17497407 -0.04826908 ...  1.5099537  -0.32151946
   0.19070287]
 [ 0.16040413  0.19577663 -0.33746952 ...  0.86385447  0.036909
   0.5219929 ]
 [-0.24866055  0.74559367  0.1431311  ...  1.3049843   0.31592712
   0.46277386]
 ...
 [ 0.53006667  0.4155307  -0.10187065 ...  1.2017733   0.27452782
  -1.0422816 ]
 [ 0.7396262   0.3290021  -0.18932408 ...  0.790695    0.23377834
  -1.2936319 ]
 [ 1.2514498   0.29501548  0.19525763 ...  0.83732474  0.02331756
  -1.4619753 ]]


In [ ]:
import pandas as pd
output = pd.DataFrame(embeddings)
print (output)

           0         1         2         3         4         5         6    \
0    -0.172000 -0.174974 -0.048269 -0.273164 -1.503079  0.360451 -0.226743   
1     0.160404  0.195777 -0.337470  0.631386 -1.019649  0.961958 -0.410149   
2    -0.248661  0.745594  0.143131  0.392315 -0.408597  1.790995 -0.497254   
3     0.019993 -0.195160  0.213574  0.338663 -0.973904  0.335602  0.103324   
4     0.542900  0.222962 -0.012518  0.622125 -0.392783  1.000384  0.109652   
...        ...       ...       ...       ...       ...       ...       ...   
7756  0.650239  0.256903 -0.051294  0.754670 -0.943332  0.127153  0.227000   
7757  0.936392  0.207242  0.039401  0.699630 -0.963015  0.183090 -0.211421   
7758  0.530067  0.415531 -0.101871  1.001012 -0.813309 -0.022368  0.106008   
7759  0.739626  0.329002 -0.189324  0.929848 -0.737598  0.183894  0.279562   
7760  1.251450  0.295015  0.195258  0.876635 -0.367716 -0.215615 -0.084943   

           7         8         9    ...       758       759    

In [ ]:
output.to_csv("/content/drive/My Drive/Seguimiento-Lucía/borradores/embeddings de literales/embedding_plats.csv", index=False)

In [ ]:
!pip install datasets

In [ ]:
import torch
from datasets import load_dataset

In [ ]:
plats_embeddings = load_dataset('lxi19/dataset_embedding_plats', data_files='embedding_plats.csv')
dataset_embeddings = torch.from_numpy(plats_embeddings["train"].to_pandas().to_numpy()).to(torch.float)

embedding_plats.csv:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
#comparación con gastronomia japonesa
jap = "/content/drive/My Drive/Seguimiento-Lucía/borradores/embeddings de literales/literales_jap_bn.txt"
# Leer el archivo de texto y cargar cada línea en una lista
with open(jap, "r", encoding="utf-8") as file:
    lineas_texto = file.readlines()

# Eliminar los caracteres de nueva línea al final de cada línea
lineas_texto = [linea.strip() for linea in lineas_texto]


In [ ]:
#embeddings
line_embeddings = model.encode(lineas_texto)

# Si deseas ver los embeddings generados:
print(line_embeddings)

[[ 0.37149912  0.07574565  0.25524393 ...  0.06301118  0.08556313
   0.1643149 ]
 [-0.03230329 -0.1447479  -0.05211821 ...  1.00891    -0.164559
   0.7175842 ]
 [ 0.26892298  0.08705245 -0.37970868 ...  0.559251    0.04791727
   0.4627625 ]
 ...
 [ 0.25547972 -0.7606288   0.6540655  ...  0.21219304 -0.24347055
   0.18186699]
 [ 0.25721174  0.01435628  0.21081297 ...  0.19724332  0.2826623
  -0.44221005]
 [ 0.3442216   0.4236121   0.24149245 ...  0.05103007  0.34010035
  -0.07411189]]


In [ ]:
jap_embeddings = torch.from_numpy(line_embeddings).to(torch.float)

In [ ]:
import torch
import torch.nn.functional as F

# dataset1_embeddings tiene tamaño [num_items1, embedding_size]
# dataset2_embeddings tiene tamaño [num_items2, embedding_size]

# Normalizar los embeddings para hacer el cálculo de similitud coseno
dataset1_norm = F.normalize(dataset_embeddings, p=2, dim=1)  # Normalizar filas en dataset1
dataset2_norm = F.normalize(jap_embeddings, p=2, dim=1)  # Normalizar filas en dataset2

# Calcular la matriz de similitud coseno entre todos los pares
# Esto produce una matriz de tamaño [num_items1, num_items2]
cosine_similarities = torch.mm(dataset1_norm, dataset2_norm.T)  # Producto de matrices

# Imprimir las dimensiones de la matriz de similitud
print("Dimensiones de la matriz de similitud coseno:", cosine_similarities.shape)

# Imprimir las primeras similitudes
print("Similitudes coseno entre los datasets:", cosine_similarities)


Dimensiones de la matriz de similitud coseno: torch.Size([7761, 20])
Similitudes coseno entre los datasets: tensor([[0.0880, 0.3811, 0.3444,  ..., 0.1402, 0.1954, 0.2017],
        [0.1738, 0.4880, 0.4520,  ..., 0.1435, 0.3130, 0.2975],
        [0.1981, 0.4740, 0.3558,  ..., 0.1957, 0.3609, 0.3468],
        ...,
        [0.2044, 0.3726, 0.3269,  ..., 0.1470, 0.3373, 0.3679],
        [0.1240, 0.3097, 0.3194,  ..., 0.0819, 0.3073, 0.3083],
        [0.2018, 0.3144, 0.3023,  ..., 0.1670, 0.3492, 0.3887]])


In [ ]:
# Encontrar el valor máximo de similitud y su posición
max_value, max_indices = torch.max(cosine_similarities, dim=1)

# Imprimir los índices y las similitudes más altas para cada línea de dataset1
for i in range(len(max_value)):
    print(f"Línea {i} de dataset1 tiene mayor similitud con la línea {max_indices[i]} de dataset2: {max_value[i].item()}")


Streaming output truncated to the last 5000 lines.
Línea 2761 de dataset1 tiene mayor similitud con la línea 10 de dataset2: 0.41823267936706543
Línea 2762 de dataset1 tiene mayor similitud con la línea 10 de dataset2: 0.43676963448524475
Línea 2763 de dataset1 tiene mayor similitud con la línea 10 de dataset2: 0.4444471597671509
Línea 2764 de dataset1 tiene mayor similitud con la línea 10 de dataset2: 0.4083147644996643
Línea 2765 de dataset1 tiene mayor similitud con la línea 1 de dataset2: 0.26835986971855164
Línea 2766 de dataset1 tiene mayor similitud con la línea 10 de dataset2: 0.4556376039981842
Línea 2767 de dataset1 tiene mayor similitud con la línea 2 de dataset2: 0.4319111108779907
Línea 2768 de dataset1 tiene mayor similitud con la línea 10 de dataset2: 0.4861273169517517
Línea 2769 de dataset1 tiene mayor similitud con la línea 10 de dataset2: 0.5179840326309204
Línea 2770 de dataset1 tiene mayor similitud con la línea 10 de dataset2: 0.40220320224761963
Línea 2771 de dat

In [ ]:
# Máscara para valores mayores a 0.79
threshold = 0.60
mask = cosine_similarities > threshold


In [ ]:
# Obtener los índices de los valores que cumplen la condición
indices = torch.nonzero(mask, as_tuple=False)

# Imprimir los pares de índices de dataset1 y dataset2 con similitud > 0.79
for index in indices:
    i, j = index[0].item(), index[1].item()
    similarity = cosine_similarities[i, j].item()
    print(f"Línea {i} de dataset1 es similar a la línea {j} de dataset2 con similitud {similarity:.4f}")


Línea 74 de dataset1 es similar a la línea 10 de dataset2 con similitud 0.6096
Línea 456 de dataset1 es similar a la línea 10 de dataset2 con similitud 0.6175
Línea 458 de dataset1 es similar a la línea 10 de dataset2 con similitud 0.6225
Línea 459 de dataset1 es similar a la línea 10 de dataset2 con similitud 0.6612
Línea 860 de dataset1 es similar a la línea 10 de dataset2 con similitud 0.6147
Línea 943 de dataset1 es similar a la línea 10 de dataset2 con similitud 0.6060
Línea 996 de dataset1 es similar a la línea 10 de dataset2 con similitud 0.6101
Línea 997 de dataset1 es similar a la línea 10 de dataset2 con similitud 0.6494
Línea 1034 de dataset1 es similar a la línea 10 de dataset2 con similitud 0.6270
Línea 1176 de dataset1 es similar a la línea 6 de dataset2 con similitud 0.8048
Línea 1176 de dataset1 es similar a la línea 19 de dataset2 con similitud 0.7001
Línea 1350 de dataset1 es similar a la línea 10 de dataset2 con similitud 0.6018
Línea 1360 de dataset1 es similar a la